In [1]:
!pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 12.8 MB/s eta 0:00:00


In [2]:
!pip install optuna-dashboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 13.0 MB/s eta 0:00:00


In [3]:
!pip install transformers

In [4]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00


In [5]:
!pip install jsonlines

In [6]:
!pip install accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 5.8 MB/s eta 0:00:00


In [7]:
from datasets import load_dataset
from datasets import load_metric
from datasets import Dataset, DatasetDict
import accelerate
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
import jsonlines
import pandas as pd
import numpy as np

In [8]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.6 MB/s eta 0:00:00


In [9]:
import datasets

# load data
data = datasets.load_from_disk("/content/drive/MyDrive/DATA 495/data_tokenized")

In [10]:
data

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'Segmented_Transcript', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 412
    })
    test: Dataset({
        features: ['Unnamed: 0', 'Segmented_Transcript', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 14
    })
    valid: Dataset({
        features: ['Unnamed: 0', 'Segmented_Transcript', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 32
    })
})

In [11]:
# load tokenizer and model, create trainer
model_name = "/content/drive/MyDrive/DATA 495/trained_3sentiment_model"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
#model.to(device_name)
trainer = Trainer(model=model)

In [21]:
# log into hugging face hub
from huggingface_hub import notebook_login

notebook_login()

## Fine-Tune Model


In [22]:
!pip install optuna

In [23]:
# import model we just trained and saved
model_checkpoint = "/content/drive/MyDrive/DATA 495/trained_3sentiment_model"

In [24]:
# define function to reinitialize model for each pass through
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=3)

In [25]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix, f1_score
import seaborn as sns
import matplotlib.pyplot as plt


In [26]:
# define the metric we're looking to optimize (accuracy)


def compute_single_metric(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    # Calculate accuracy
    acc = accuracy_score(labels, preds)

    return {'accuracy': acc}


In [27]:
# convert it to optuna single metric
def objective(metrics):
    return metrics['accuracy']

In [28]:
# redefine training arguments
batch_size = 4
logging_steps = 1000
args = TrainingArguments( output_dir = model_name,
                                  num_train_epochs = 4,
                                  learning_rate = 2e-5,
                                  per_device_train_batch_size = batch_size,
                                  per_device_eval_batch_size = batch_size,
                                  weight_decay = 0.01,
                                  evaluation_strategy = "epoch",
                                  metric_for_best_model='accuracy',
                                  disable_tqdm = False,
                                  logging_steps = logging_steps,
                                  push_to_hub = True,
                                  log_level = "error"
                                   )

In [29]:
# define Trainer function

trainer = Trainer(
    model_init=model_init,
    args=args,
    train_dataset=data["train"],
    eval_dataset=data["valid"],
    tokenizer=tokenizer,
    compute_metrics=compute_single_metric
)

In [30]:
train_dataset = data["train"].shard(index=1, num_shards=10)

In [31]:
# optuna training function

best_run = trainer.hyperparameter_search(n_trials=10, direction="maximize")

[I 2023-11-24 22:57:50,817] A new study created in memory with name: no-name-a8c09428-07c4-42b6-97b1-d689c1f86e82


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.726668,0.375000
2,No log,0.715913,0.375000
3,No log,0.655840,0.687500


[I 2023-11-24 22:58:22,325] Trial 0 finished with value: 0.6875 and parameters: {'learning_rate': 9.166714056561446e-05, 'num_train_epochs': 3, 'seed': 36, 'per_device_train_batch_size': 32}. Best is trial 0 with value: 0.6875.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.852615,0.843750
2,No log,1.587913,0.781250
3,No log,1.691658,0.781250


[I 2023-11-24 22:58:55,987] Trial 1 finished with value: 0.78125 and parameters: {'learning_rate': 3.0700327520961375e-05, 'num_train_epochs': 3, 'seed': 2, 'per_device_train_batch_size': 8}. Best is trial 1 with value: 0.78125.


...

[I 2023-11-24 23:01:20,354] Trial 7 finished with value: 0.84375 and parameters: {'learning_rate': 1.5380994049418791e-06, 'num_train_epochs': 3, 'seed': 16, 'per_device_train_batch_size': 16}. Best is trial 3 with value: 0.875.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.437303,0.843750
2,No log,0.445727,0.843750


[I 2023-11-24 23:01:40,386] Trial 8 finished with value: 0.84375 and parameters: {'learning_rate': 1.558464611244768e-06, 'num_train_epochs': 2, 'seed': 20, 'per_device_train_batch_size': 32}. Best is trial 3 with value: 0.875.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.525071,0.843750
2,No log,0.503936,0.812500


[I 2023-11-24 23:02:00,372] Trial 9 finished with value: 0.8125 and parameters: {'learning_rate': 1.903265990043712e-05, 'num_train_epochs': 2, 'seed': 32, 'per_device_train_batch_size': 32}. Best is trial 3 with value: 0.875.


In [32]:
best_run

BestRun(run_id='3', objective=0.875, hyperparameters={'learning_rate': 7.369177356718335e-06, 'num_train_epochs': 3, 'seed': 29, 'per_device_train_batch_size': 32}, run_summary=None)